In [1]:
!pip install catboost

     |████████████████████████████████| 65.7MB 44kB/s 


In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission_QrCyCoT.csv to sample_submission_QrCyCoT.csv
Saving test_YCcRUnU.csv to test_YCcRUnU.csv
Saving train_Df64byy.csv to train_Df64byy.csv


In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [4]:
train_df = pd.read_csv('/content/train_Df64byy.csv')
test_df = pd.read_csv('/content/test_YCcRUnU.csv')

In [5]:
def make_df_ready(df):
  df = df.set_index('ID')
  df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].replace('14+', '15.0').astype(float)
  df['Is_Spouse'] = df['Is_Spouse'].map({'No': 1, 'Yes': 0})
  df['Accomodation_Type'] = df['Accomodation_Type'].map({'Owned': 1, 'Rented': 0})
  df['Reco_Insurance_Type'] = df['Reco_Insurance_Type'].map({'Individual': 1, 'Joint': 0})
  df['Holding_Policy_Type'] = df['Holding_Policy_Type'].astype(str)
  df['Reco_Policy_Cat'] = df['Reco_Policy_Cat'].astype(str)
  df['Region_Code'] = df['Region_Code'].astype(str)
  df['Health Indicator'] = df['Health Indicator'].astype(str)
  return df

In [6]:
df_ready_train = make_df_ready(train_df)

In [141]:
X = df_ready_train.loc[:, df_ready_train.columns != 'Response']
y = df_ready_train.loc[:, 'Response']
# X = X[imp_features_96]
cat_features=['City_Code', 'Region_Code', 'Health Indicator', 'Reco_Policy_Cat']

In [174]:
params = dict(task_type='GPU',  
iterations=1000,
objective='Logloss', 
random_seed=21,
learning_rate=1,
eval_metric='AUC:hints=skip_train~false')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, shuffle=True)

cv_data = X_train.copy()
labels = y_train.values

cv_dataset = Pool(data=cv_data, 
                  label=labels, 
                  cat_features=cat_features)

# scores = cv(cv_dataset,
#             params,
#             fold_count=10, seed=21, partition_random_seed=21,
#             early_stopping_rounds=20)

In [175]:
model = CatBoostClassifier(**params)

In [176]:
grid = {
        'learning_rate': [0.1, 0.3, 0.5, 0.8],
        'depth': [4, 6, 8, 10]
        }

In [177]:
gs_res = model.grid_search(param_grid = grid,
                  X=cv_dataset,
                  cv=10,
                  partition_random_seed=21,
                  train_size=0.9,
        )

bestTest = 0.7901728153
bestIteration = 545
0:	loss: 0.7901728	best: 0.7901728 (0)	total: 32.4s	remaining: 8m 5s
bestTest = 0.7899057865
bestIteration = 115
1:	loss: 0.7899058	best: 0.7901728 (0)	total: 1m 4s	remaining: 7m 34s
bestTest = 0.787479043
bestIteration = 24
2:	loss: 0.7874790	best: 0.7901728 (0)	total: 1m 37s	remaining: 7m 2s
bestTest = 0.7858130634
bestIteration = 21
3:	loss: 0.7858131	best: 0.7901728 (0)	total: 2m 10s	remaining: 6m 30s
bestTest = 0.791246295
bestIteration = 939
4:	loss: 0.7912463	best: 0.7912463 (4)	total: 2m 52s	remaining: 6m 19s
bestTest = 0.7901301682
bestIteration = 57
5:	loss: 0.7901302	best: 0.7912463 (4)	total: 3m 34s	remaining: 5m 57s
bestTest = 0.7881249785
bestIteration = 36
6:	loss: 0.7881250	best: 0.7912463 (4)	total: 4m 16s	remaining: 5m 30s
bestTest = 0.785441488
bestIteration = 29
7:	loss: 0.7854415	best: 0.7912463 (4)	total: 5m	remaining: 5m
bestTest = 0.7907491326
bestIteration = 501
8:	loss: 0.7907491	best: 0.7912463 (4)	total: 5m 51s	rem

In [178]:
pd.DataFrame(gs_res['cv_results'])

,iterations,test-AUC-mean,test-AUC-std,train-AUC-mean,train-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.717611,0.015642,0.717042,0.008294,0.649763,0.003709,0.646848,0.000898
1,1,0.738461,0.020637,0.732646,0.010989,0.615020,0.003257,0.611061,0.001646
2,2,0.750480,0.011542,0.739205,0.005049,0.587099,0.004769,0.583104,0.001569
3,3,0.755510,0.009947,0.745472,0.003637,0.565348,0.004579,0.560712,0.001653
4,4,0.757304,0.008918,0.748386,0.002108,0.547017,0.005075,0.543322,0.001656
...,...,...,...,...,...,...,...,...,...
995,995,0.793062,0.008760,0.804738,0.001713,0.445791,0.006886,0.434736,0.001378
996,996,0.793065,0.008757,0.804742,0.001713,0.445792,0.006885,0.434733,0.001379
997,997,0.793064,0.008756,0.804742,0.001713,0.445793,0.006884,0.434733,0.001379
998,998,0.793071,0.008758,0.804744,0.001715,0.445785,0.006887,0.434731,0.001381


In [181]:
params_to_use = params.copy()
params_to_use.update(gs_res['params'])

In [184]:
model_2 = CatBoostClassifier(**params_to_use)

In [188]:
model_2.fit(X_train, y_train, cat_features=cat_features)

0:	learn: 0.7302081	total: 397ms	remaining: 6m 36s
1:	learn: 0.7439361	total: 454ms	remaining: 3m 46s
2:	learn: 0.7491387	total: 538ms	remaining: 2m 58s
3:	learn: 0.7491140	total: 572ms	remaining: 2m 22s
4:	learn: 0.7541735	total: 617ms	remaining: 2m 2s
5:	learn: 0.7532991	total: 672ms	remaining: 1m 51s
6:	learn: 0.7553161	total: 726ms	remaining: 1m 42s
7:	learn: 0.7561777	total: 780ms	remaining: 1m 36s
8:	learn: 0.7568705	total: 813ms	remaining: 1m 29s
9:	learn: 0.7575198	total: 949ms	remaining: 1m 33s
10:	learn: 0.7583832	total: 1.03s	remaining: 1m 32s
11:	learn: 0.7592122	total: 1.26s	remaining: 1m 44s
12:	learn: 0.7591599	total: 1.36s	remaining: 1m 43s
13:	learn: 0.7595535	total: 1.4s	remaining: 1m 38s
14:	learn: 0.7634802	total: 1.44s	remaining: 1m 34s
15:	learn: 0.7661835	total: 1.48s	remaining: 1m 31s
16:	learn: 0.7694217	total: 1.52s	remaining: 1m 28s
17:	learn: 0.7711879	total: 1.6s	remaining: 1m 27s
18:	learn: 0.7722831	total: 1.66s	remaining: 1m 25s
19:	learn: 0.7729125	tota

In [189]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [209]:
train_pred_p = model.predict_proba(X_train)
test_pred_p = model.predict_proba(X_test)

In [251]:
thresh = 0.42

print(f'Train: {roc_auc_score(y_train, (train_pred_p[:, 1] > thresh).astype(int))}')
print(f'Test: {roc_auc_score(y_test, (test_pred_p[:, 1] > thresh).astype(int))}')

Train: 0.7877436130188513
Test: 0.6852797586487289


In [252]:
ready_test = make_df_ready(test_df)

In [253]:
submission_df = pd.read_csv('/content/sample_submission_QrCyCoT.csv')

In [254]:
submission_df['Response'] = ((model_2.predict_proba(ready_test))[:, 1] > thresh).astype(int)

In [255]:
train_df['Response'].value_counts(normalize=True)

0    0.760053
1    0.239947
Name: Response, dtype: float64

In [257]:
submission_df['Response'].value_counts(normalize=True)

0    0.761752
1    0.238248
Name: Response, dtype: float64

In [258]:
submission_df.to_csv('submission 3_56, thresh point42.csv', index=False)